### Convert h5 to frozen graph

##### Tested this code on TF 1.15 and TF 2.2, none of them worked fr me.

In [13]:
# force reset ipython namespaces
%reset -f

import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from tensorflow import keras


import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import nn
from tensorflow.keras.backend import shape
from tensorflow.keras.layers import Dropout


import efficientnet.tfkeras
from tensorflow.keras.models import load_model


from tensorflow.python.framework import graph_io

import segmentation_models as sm




# Clear any previous session.
tf.keras.backend.clear_session()

save_pb_dir = '/home/mohan/git/segmentation_models/examples/save-pb/'
model_fname = '/home/mohan/git/segmentation_models/examples/best_mode_model_filel.h5'


#model_fname = './model/tf_model_base_model_2602.pb'
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model_ss_plf.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

# This line must be executed before loading Keras model.
tf.keras.backend.set_learning_phase(0) 

class FixedDropout(Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        return tuple([shape(inputs)[i] if sh is None else sh for i, sh in enumerate(self.noise_shape)])

customObjects = {
    'swish': nn.swish,
    'FixedDropout': FixedDropout,
    'dice_loss_plus_1binary_focal_loss' : sm.losses.binary_focal_dice_loss,
    'iou_score': sm.metrics.iou_score,
    'f1-score': sm.metrics.f1_score
}

model = load_model(model_fname, customObjects)

##Custom code for loading the model ends here

session = tf.keras.backend.get_session()

input_names = [t.op.name for t in model.inputs]
output_names = [t.op.name for t in model.outputs]

# Prints input and output nodes names, take notes of them.
print(input_names, output_names)

frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

['input_1'] ['sigmoid/Sigmoid']
INFO:tensorflow:Froze 546 variables.
INFO:tensorflow:Converted 546 variables to const ops.


### Convert into tensor RT format

In [15]:
import tensorflow.contrib.tensorrt as trt

trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Running against TensorRT version 0.0.0


### Save the tensor RT graph

In [16]:
graph_io.write_graph(trt_graph, "./converted_trt_graph/",
                     "trt_graph_ss_model.pb", as_text=False)

'./converted_trt_graph/trt_graph_ss_model.pb'

In [ ]:
### Load the new model

In [ ]:
trt_model_fname = './converted_trt_graph/'


# We need to create an SSDLoss object in order to pass that to the model loader.
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

#K.clear_session() # Clear previous models from memory.

#trt_model = tf.keras.models.load_model(trt_model_fname, custom_objects={'AnchorBoxes': AnchorBoxes,'compute_loss': ssd_loss.compute_loss})
trt_model = tf.compat.v1.saved_model.loader.load(trt_model_fname)

In [ ]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)

tf.import_graph_def(graph_def, name='')

In [ ]:
tf_input = tf_sess.graph.get_tensor_by_name('input_1:0')
print(tf_input)
tf_predictions = tf_sess.graph.get_tensor_by_name('predictions/concat:0')
print(tf_predictions)

In [ ]:
IMAGE_PATH = './1478899159823020309.jpg'

In [17]:
!pwd

/home/mohan/git/segmentation_models/examples


### test

In [18]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image

pb_model_dir = "./converted_trt_graph"
h5_model = "./trt_model.h5"

# Loading the Tensorflow Saved Model (PB)
model = tf.keras.models.load_model(pb_model_dir)
print(model.summary())

# Saving the Model in H5 Format
tf.keras.models.save_model(model, h5_model)

# Loading the H5 Saved Model
loaded_model_from_h5 = tf.keras.models.load_model(h5_model)
print(loaded_model_from_h5.summary())

OSError: SavedModel file does not exist at: ./converted_trt_graph/{saved_model.pbtxt|saved_model.pb}